In [1]:
import os

PROJECT_DIR = ''

IS_GOOGLE_COLAB_ENABLED = False

if IS_GOOGLE_COLAB_ENABLED:
    print('Google Colab is enabled. Running on Google Colab.')

    from google.colab import drive
    # Mount Google Drive
    drive.mount('/content/drive', force_remount=True)

    PROJECT_DIR = '/content/drive/MyDrive/Colab/'

    # Ensure the directory exists
    os.makedirs(PROJECT_DIR, exist_ok=True)

else:
    print('Google Colab is not enabled. Running locally.')

    # Local configuration and folder setup
    PROJECT_DIR = '/Users/ondergormez/Repos/BLM5109_Collective_Learning/03-Project/'

LLM_ANSWERS_DIR = PROJECT_DIR + 'llm_answers'
RAGAS_SCORES_DIR = PROJECT_DIR + 'ragas_scores'

# Ensure the directory exists
os.makedirs(LLM_ANSWERS_DIR, exist_ok=True)
os.makedirs(RAGAS_SCORES_DIR, exist_ok=True)

Google Colab is not enabled. Running locally.


# Logger Kullanımı

- Oluşturulan kod içerisinde loglama işlemleri için `logging` kütüphanesi kullanılmıştır.
- Böylelikle geliştirme aşamasında debug için koyulan kodların çıktılarına bakılarak hata ayıklama işlemleri yapılabilir. Bu kodların çıktıları rapor sunulurken kaldırılacaktır.


In [2]:
import logging
import logging.config
import yaml
import textwrap


class CustomFormatter(logging.Formatter):

    def format(self, record):
        wrapped_message = super().format(record)

        # wrap the log messages to 120 characters to better fit the screen
        if (len(wrapped_message) > 120):
            wrapped_message = "\n".join(textwrap.wrap(wrapped_message, width=120))

            # indent the wrapped lines but not the first line
            first_line, rest = wrapped_message.split('\n', 1)
            wrapped_message = first_line + '\n' + textwrap.indent(rest, ' ' * 4) + '\n'

        return wrapped_message


LOGGER_CONFIG_FILE = PROJECT_DIR + 'logger_config.yaml'

# Load the YAML configuration
with open(LOGGER_CONFIG_FILE, 'r') as file:
    config = yaml.safe_load(file.read())
    logging.config.dictConfig(config)

# Apply the custom formatter to the selected handler
logger = logging.getLogger("default_logger")
for handler in logger.handlers:
    if isinstance(handler, logging.StreamHandler):
        handler.setFormatter(CustomFormatter(handler.formatter._fmt, datefmt=handler.formatter.datefmt))

# TODO: Set the log level to INFO
# logger.setLevel(logging.DEBUG)
logger.setLevel(logging.INFO)


logger.debug("This is a debug message that will be wrapped after 140 characters. " * 10)
logger.info("Logging is configured successfully.")

[INFO] [2025-01-28T00:47:17.693Z] Logging is configured successfully.


In [3]:
cosmos_model_name = "cosmos_dpo"
cosmos_llm_answers_task_b_file_names = [
    '2025_01_27-13_23_49_cosmos_dpo_llm_answers_for_cntx_length_15_and_position_of_correct_answer_1.csv',
    '2025_01_27-13_27_58_cosmos_dpo_llm_answers_for_cntx_length_15_and_position_of_correct_answer_8.csv',
    '2025_01_27-13_32_15_cosmos_dpo_llm_answers_for_cntx_length_15_and_position_of_correct_answer_15.csv',
]

cosmos_llm_answers_task_b_file_names

['2025_01_27-13_23_49_cosmos_dpo_llm_answers_for_cntx_length_15_and_position_of_correct_answer_1.csv',
 '2025_01_27-13_27_58_cosmos_dpo_llm_answers_for_cntx_length_15_and_position_of_correct_answer_8.csv',
 '2025_01_27-13_32_15_cosmos_dpo_llm_answers_for_cntx_length_15_and_position_of_correct_answer_15.csv']

In [4]:
import time


def save_scores(llm_answers_file_name, llm_answers_df):
    # add time stamp to each file name for unique file names and further analysis
    time_stamp = time.strftime("%Y_%m_%d-%H_%M_%S")

    file_name = f'{RAGAS_SCORES_DIR}/{llm_answers_file_name}_{time_stamp}_ragas_scores.csv'

    llm_answers_df.to_csv(file_name, index=True)
    logger.info(f"Scores are saved to the file: '{file_name}'")

    return file_name

In [5]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_correctness, answer_relevancy
import pandas as pd

import os

# TODO: Replace the api key with the correct one from https://platform.openai.com/settings/organization/api-keys
# os.environ["OPENAI_API_KEY"] = "Your API Key"


def get_ragas_scores(file_names):
    ragas_scores_file_names = []

    model_name_for_ragas_score_eval = ''
    for file_name in file_names:
        logger.info(f"Evaluating ragas score for the file: {file_name}")

        llm_answers_df = pd.read_csv(LLM_ANSWERS_DIR + '/' + file_name)
        logger.info("Shape of the dataframe: " + str(llm_answers_df.shape))
        print()

        # Add new columns to store the metrics
        llm_answers_df[f'faithfulness'] = ""
        llm_answers_df['answer_correctness'] = ""
        llm_answers_df['answer_relevancy'] = ""

        if 'cosmos' in file_name:
            model_name_for_ragas_score_eval = cosmos_model_name
        elif 'gemma' in file_name:
            model_name_for_ragas_score_eval = gemma_model_name

        for i in range(len(llm_answers_df)):
            data_sample = {
                'question': [
                    llm_answers_df.iloc[i]['question'],
                ],
                'ground_truth': [
                    llm_answers_df.iloc[i]['answer'],
                ],
                'answer': [
                    llm_answers_df.iloc[i][f'{model_name_for_ragas_score_eval}_answer'],
                ],
                'contexts': [
                    [
                        llm_answers_df.iloc[i]['context'],
                    ],
                ]
            }

            dataset = Dataset.from_dict(data_sample)
            score = evaluate(dataset, metrics=[faithfulness, answer_correctness, answer_relevancy])
            logger.info(f"Scores for the question {i + 1} of {len(llm_answers_df)}\n: {score}")

            score_df = score.to_pandas()

            chunk_unique_id = llm_answers_df.iloc[i]['id']
            index_of_chunk = llm_answers_df['id'] == chunk_unique_id

            llm_answers_df.loc[index_of_chunk, 'faithfulness'] = score_df.iloc[0].get('faithfulness')
            llm_answers_df.loc[index_of_chunk, 'answer_correctness'] = score_df.iloc[0].get('answer_correctness')
            llm_answers_df.loc[index_of_chunk, 'answer_relevancy'] = score_df.iloc[0].get('answer_relevancy')

        # get just file name without path and extension
        temp_fn = os.path.splitext(os.path.basename(file_name))[0]

        fn = save_scores(temp_fn, llm_answers_df)
        ragas_scores_file_names.append(fn)

    return ragas_scores_file_names

In [6]:
ragas_scores_file_names = get_ragas_scores(cosmos_llm_answers_task_b_file_names)

[INFO] [2025-01-28T00:47:18.643Z] Evaluating ragas score for the file:
    2025_01_27-13_23_49_cosmos_dpo_llm_answers_for_cntx_length_15_and_position_of_correct_answer_1.csv

[INFO] [2025-01-28T00:47:18.651Z] Shape of the dataframe: (50, 11)



Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

[INFO] [2025-01-28T00:47:45.810Z] Scores for the question 1 of 50 : {'faithfulness': 0.8889, 'answer_correctness':
    0.3223, 'answer_relevancy': 0.7940}



Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

[INFO] [2025-01-28T00:48:05.867Z] Scores for the question 2 of 50 : {'faithfulness': 0.4000, 'answer_correctness':
    0.6134, 'answer_relevancy': 0.8788}



...

[INFO] [2025-01-28T01:38:39.328Z] Scores for the question 48 of 50 : {'faithfulness': 0.5000, 'answer_correctness':
    0.7285, 'answer_relevancy': 0.8149}



Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

[INFO] [2025-01-28T01:38:58.614Z] Scores for the question 49 of 50 : {'faithfulness': 0.6667, 'answer_correctness':
    0.5792, 'answer_relevancy': 0.8355}



Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

[INFO] [2025-01-28T01:39:24.100Z] Scores for the question 50 of 50 : {'faithfulness': 1.0000, 'answer_correctness':
    0.9716, 'answer_relevancy': 0.0000}

[INFO] [2025-01-28T01:39:24.117Z] Scores are saved to the file: '/Users/ondergormez/Repos/BLM5109_Collective_Learning/03
    -Project/ragas_scores/2025_01_27-
    13_32_15_cosmos_dpo_llm_answers_for_cntx_length_15_and_position_of_correct_answer_15_2025_01_28-
    01_39_24_ragas_scores.csv'



In [7]:
ragas_scores_file_names

['/Users/ondergormez/Repos/BLM5109_Collective_Learning/03-Project/ragas_scores/2025_01_27-13_23_49_cosmos_dpo_llm_answers_for_cntx_length_15_and_position_of_correct_answer_1_2025_01_28-01_10_30_ragas_scores.csv',
 '/Users/ondergormez/Repos/BLM5109_Collective_Learning/03-Project/ragas_scores/2025_01_27-13_27_58_cosmos_dpo_llm_answers_for_cntx_length_15_and_position_of_correct_answer_8_2025_01_28-01_24_55_ragas_scores.csv',
 '/Users/ondergormez/Repos/BLM5109_Collective_Learning/03-Project/ragas_scores/2025_01_27-13_32_15_cosmos_dpo_llm_answers_for_cntx_length_15_and_position_of_correct_answer_15_2025_01_28-01_39_24_ragas_scores.csv']